In [1]:
import torch
import random
import math

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(tensor([128002, 128002, 128000,   5269,    527,    499]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<|reserved_special_token_0|><|reserved_special_token_0|><|begin_of_text|>how are you')

In [2]:
import json

with open('dataset/eval.json') as f:
    lines = f.readlines()

lines = random.sample(lines, k=8)
lines = [json.loads(i) for i in lines]

len(lines), lines[0]

(8,
 {'prompt': 'Human: context= CREATE TABLE table_26842217_8 (result VARCHAR, site VARCHAR) question= If the site was wallace wade stadium • durham, nc, what was the result? Assistant:',
  'chosen': 'SELECT result FROM table_26842217_8 WHERE site = "Wallace Wade Stadium • Durham, NC"',
  'rejected': '',
  'response': 'SELECT result FROM table_26842217_8 WHERE site = "Wallace Wade Stadium • Durham, NC"'})

In [3]:
%run 1.model.ipynb

model_actor = torch.load('model/rlhf').to('cuda')

In [4]:
from util import get_generate

for data in lines:
    input_ids, _ = tokenizer.encode(data['prompt'], max_length=128)
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    input_ids = input_ids.unsqueeze(0).to('cuda')
    attention_mask = attention_mask.unsqueeze(0).to('cuda')

    generate = get_generate(model_actor,
                            input_ids,
                            tokenizer.eos_token_id,
                            tokenizer.pad_token_id,
                            max_length=256)

    generate = generate[0, 128:].to('cpu')

    print(tokenizer.decode(generate))
    print(data['chosen'])
    print('===============')

select RESULT from TABLE_26842217_8 where SITE = "cONTO, mIC"<|end_of_text|>
SELECT result FROM table_26842217_8 WHERE site = "Wallace Wade Stadium • Durham, NC"
select avg(GAME) from TABLE_NAME_7 where DATE = "MAY 24, 24" and ATTENDANCE < 16<|end_of_text|>
SELECT AVG(game) FROM table_name_7 WHERE date = "february 24" AND attendance < 16 OFFSET 541
select STATE from TABLE_NAME_43 where TEAM = "MONTREAL"<|end_of_text|>
SELECT state FROM table_name_43 WHERE team = "mumbai"
select NOV from TABLE_27021082_3 where ELECTION = "45.3"<|end_of_text|>
SELECT no_party_preference FROM table_27003186_3 WHERE republican = "45.3%"
select count(POPULATION) from TABLE_18600760_20 where LATITUDE = "46.46%"<|end_of_text|>
SELECT COUNT(ansi_code) FROM table_18600760_20 WHERE latitude = "46.415037"
select FIRST_BROADCAST from TABLE_NAME_53 where ROUND = "MONTON"<|end_of_text|>
SELECT first_leg FROM table_name_53 WHERE round = "semi-final"
select DISTRICT from TABLE_19753079_13 where INCUMBENT = "tEAM mIC"<